In [ ]:
!pip install nlpaug
!pip install transformers
!pip install torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import pandas as pd
import nlpaug.augmenter.word as naw
from collections import Counter


In [ ]:
df = pd.read_csv("/content/transcribed.csv")  # replace with your filename
df.head()

,filename,transcription,label
0,audio_90.wav,A situation in which there is a lot of compet...,3.0
1,audio_581.wav,"All right, I would say my goal, which I would...",3.0
2,audio_77.wav,All right. A crowded market is where lots of ...,2.5
3,audio_817.wav,My favorite hobby is playing cricket. I like ...,2.0
4,audio_694.wav,It is where a lot of money has come in to sto...,5.0


In [ ]:
print(df['label'].value_counts().sort_index())


label
1.0      1
1.5      3
2.0     70
2.5     40
3.0     87
3.5     23
4.0     52
4.5     58
5.0    110
Name: count, dtype: int64


In [ ]:
import nlpaug.augmenter.word as naw

# Use GPU if available
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased',
    action="substitute",
    device='cpu'  # or 'cpu' if no GPU
)


In [ ]:
# Drop rows with missing transcription or label
df = df.dropna(subset=['transcription', 'label'])

# Convert transcription column to string type
df['transcription'] = df['transcription'].astype(str)


In [ ]:
from collections import defaultdict

target_count = 200 # Desired samples per label
augmented_rows = []

label_counts = df['label'].value_counts()

for label, count in label_counts.items():
    if count >= target_count:
        continue

    needed = target_count - count
    reps = (needed // count) + 1

    subset = df[df['label'] == label]

    new_id_counter = 0
    for _, row in subset.iterrows():
        for _ in range(reps):
            try:
                aug_text = aug.augment(row['transcription'])
                new_filename = f"{row['filename'].split('.')[0]}_aug{new_id_counter}.wav"
                new_id_counter += 1
                augmented_rows.append({
                    'filename': new_filename,
                    'transcription': aug_text,
                    'label': row['label']
                })
                if len([r for r in augmented_rows if r['label'] == label]) >= needed:
                    break
            except Exception as e:
                print(f"Augmentation error: {e}")


In [ ]:
aug_df = pd.DataFrame(augmented_rows)
df_final = pd.concat([df, aug_df]).sample(frac=1).reset_index(drop=True)

print(df_final['label'].value_counts())


label
2.0    138
4.5    116
5.0    109
4.0    104
2.5     98
3.0     87
3.5     82
1.0     80
1.5     80
Name: count, dtype: int64
